In [2]:
import pandas as pd


In [3]:
df = pd.read_excel(r'C:\Users\Harshpal Juneja\OneDrive\Desktop\docs_req.xlsx')

In [4]:
df.head()

,Staff ID,Name,Home Pin,Branch Pin
0,214,Ramesh Kumar Yadav,231001,854301
1,715,Rahul Shukla,228171,246763
2,1207,Deshraj Singh,246749,302028
3,1243,Gaurav Sharma,203131,202001
4,1436,Rahul Kumar,110044,245304


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Staff ID    1898 non-null   int64 
 1   Name        1898 non-null   object
 2   Home Pin    1898 non-null   int64 
 3   Branch Pin  1898 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 59.4+ KB


In [84]:
from connection import create_connection, fetch_data_with_query

# Call create_connection to get a connection object
conn = create_connection()

# Define your SQL query
custom_sql_query = """

select s.staffid,staffadd,bc.branchid,bc.latitude, bc.longitude
from staff s
join branchmaster bc
on s.branchid= bc.branchid

"""

# Fetch data based on the custom SQL query
branch_map = fetch_data_with_query(custom_sql_query)




C:\Users\Harshpal Juneja\connection.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [86]:
branch_map.head()

,staffid,staffadd,branchid,latitude,longitude
0,9,A-2 PURNARWAS BHALSWA COLONY DELHI-42,1,28.768107,77.175602
1,15,BHALSWA DELHI-42,1,28.768107,77.175602
2,17,BHALSWA,1,28.768107,77.175602
3,18,C-5/270 J.J.COLONY BHALSWA DELHI-42,1,28.768107,77.175602
4,29,A-2/839 J.J.COLONY BHALSWA DAIRY NEW DELHI-42,1,28.768107,77.175602


In [2]:
branch_map.rename(columns={'latitude': 'branch_latitude', 'longitude': 'branch_longitude'}, inplace=True)



In [88]:
branch_map.head()

,staffid,staffadd,branchid,branch_latitude,branch_longitude
0,9,A-2 PURNARWAS BHALSWA COLONY DELHI-42,1,28.768107,77.175602
1,15,BHALSWA DELHI-42,1,28.768107,77.175602
2,17,BHALSWA,1,28.768107,77.175602
3,18,C-5/270 J.J.COLONY BHALSWA DELHI-42,1,28.768107,77.175602
4,29,A-2/839 J.J.COLONY BHALSWA DAIRY NEW DELHI-42,1,28.768107,77.175602


In [89]:
df = pd.merge(df,branch_map,left_on='Staff ID',right_on='staffid',how='left')

In [90]:
df.head()

,Staff ID,Name,Home Pin,Branch Pin,staffid,staffadd,branchid,branch_latitude,branch_longitude
0,214,Ramesh Kumar Yadav,231001,854301,214,"VILLAGE NEWARHIA, POST KATHANAI, DISTRICT MIRZ...",383,25.766282,87.482993
1,715,Rahul Shukla,228171,246763,715,"VILL - MARUI KRISHNDASPUR,POST -PAUDHAN RAMPUR...",34,29.603513,78.341156
2,1207,Deshraj Singh,246749,302028,1207,"VILL: MOHD AMI KHANPUR NAYA GAON, POST : BIJNO...",466,26.999641,75.881002
3,1243,Gaurav Sharma,203131,202001,1243,"H.NO.49 NAVALPURA, KALRALA KHURJADISS: BULANDS...",17,27.889674,78.046579
4,1436,Rahul Kumar,110044,245304,1436,"HOUSE NO.440,GALI NO.13 LAKHPAT COLONY PART-1 ...",72,28.705630,77.652107


In [91]:
df.drop(['staffid','branchid','Name'],axis=1,inplace=True)

In [92]:
df[df['Staff ID']== 20751]

,Staff ID,Home Pin,Branch Pin,staffadd,branch_latitude,branch_longitude
682,20751,246738,244231,"Gajipur, Khaspura, Chandpur, Bijnor, Uttar Pra...",28.969017,78.252676


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Staff ID          1898 non-null   int64  
 1   Home Pin          1898 non-null   int64  
 2   Branch Pin        1898 non-null   int64  
 3   staffadd          1898 non-null   object 
 4   branch_latitude   1898 non-null   float64
 5   branch_longitude  1898 non-null   float64
dtypes: float64(2), int64(3), object(1)
memory usage: 89.1+ KB


In [130]:
import requests
import pandas as pd

def get_lat_long_from_google_api(postal_code, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={postal_code}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            lat = data['results'][0]['geometry']['location']['lat']
            lng = data['results'][0]['geometry']['location']['lng']
            return lat, lng
    return None, None

# Your Google Maps API Key
google_api_key = ''



# Define lists to store latitude and longitude values
latitudes = []
longitudes = []

# Loop through each postal code and get latitude and longitude
for postal_code in df['Home Pin']:
    lat, lon = get_lat_long_from_google_api(postal_code, google_api_key)
    latitudes.append(lat)
    longitudes.append(lon)

# Add latitude and longitude columns to the DataFrame
df['Home_Pin_Latitude'] = latitudes
df['Home_Pin_Longitude'] = longitudes



In [94]:
# from time import sleep
import time
api_key = ''


In [3]:
# Distance Calculation

In [20]:
from math import radians, sin, cos, sqrt, atan2
import pandas as pd


def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_of_earth = 6371  # Radius of the Earth in kilometers
    distance = radius_of_earth * c

    return distance

# Calculate distances for each row in the DataFrame
distances = []
for index, row in df.iterrows():
    home_lat = row['Home Latitude']
    home_lon = row['Home Longitude']
    branch_lat = row['Branch Latitude']
    branch_lon = row['Branch Longitude']
    distance_km = haversine(home_lat, home_lon, branch_lat, branch_lon)
    distances.append(distance_km)

# Add the distances as a new column to the DataFrame
df['Distance (km)'] = distances

In [114]:
df.isnull().sum()

Staff ID             0
Home Pin             0
Branch Pin           0
staffadd             0
home_latitude        3
home_longitude       3
branch_latitude     15
branch_longitude    15
Distance (km)       16
dtype: int64

In [144]:
# df.to_excel(r'C:\Users\Harshpal Juneja\OneDrive\Desktop\distance_between_pincodes.xlsx',index=False)

In [28]:
filtered_df = df[~df[['Distance (km)', 'Branch Longitude']].isnull().any(axis=1)]


In [29]:
filtered_df.shape

(1715, 9)

In [70]:
df.isnull().sum()

Staff ID             0
Home Pin             0
Branch Pin           0
staffadd             0
branch_latitude      0
branch_longitude     0
home_latitude       10
home_longitude      10
Distance (km)       10
dtype: int64

In [117]:
# df.to_excel(r'C:\Users\Harshpal Juneja\OneDrive\Desktop\distance_pincode_wise.xlsx')